In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyNN.nest as p
import relu_utils as alg
import spiking_relu as sr
import random
import mnist_utils as mu
import os.path
import sys

In [61]:
w_listf = 'scaled_weight'
dbn_f = 'b10_epoc5'
dbnet = alg.load_dict(dbn_f)
cell_params_lif = {'cm': 0.25,
                   'i_offset': 0.0,
                   'tau_m': 20.0,
                   'tau_refrac': 1.,
                   'tau_syn_E': 1.0,
                   'tau_syn_I': 1.0,
                   'v_reset': -70.0,
                   'v_rest': -65.0,
                   'v_thresh': -50.0
                   }

In [62]:
if os.path.isfile('%s.pkl'%w_listf):
    scaled_w = alg.load_dict(w_listf)
    w = scaled_w['w']
    k = scaled_w['k']
    x0 = scaled_w['x0']
    y0 = scaled_w['y0']
    print 'found w_list file'
else:
    w, k, x0, y0 = sr.w_adjust(dbnet, cell_params_lif)
    scaled_w = {}
    scaled_w['w'] = w
    scaled_w['k'] = k
    scaled_w['x0'] = x0
    scaled_w['y0'] = y0
    alg.save_dict(scaled_w, w_listf)

found w_list file


In [66]:
test = dbnet['test_x']
SUM_rate = 2000.
for i in range(test.shape[0]):
    test[i] = test[i] / sum(test[i]) * SUM_rate
test_in = test#[0,:]
for w in scaled_w['w']:
    out = alg.ReLU(test_in, w)
    out = sr.transf(scaled_w['k'], scaled_w['x0'], scaled_w['y0'], out/1000.)
    out[out<0] = 0
    test_in = out
#print np.transpose(out.astype(int))
label = out
predict = np.argmax(label, axis=1)
index = np.where(label.max(axis=1)==0)[0]
predict[index] = -1
result = predict == dbnet['test_y']
result = result.astype(int)
result[index] = -1

In [69]:
print np.where(result==1)[0].shape, np.where(result==0)[0].shape, np.where(result==-1)[0].shape

(8958,) (1032,) (9,)


In [57]:
test_x = dbnet['test_x']
offset = 0
random.seed(2)
num_test = 10
test = test_x[offset:(offset+num_test), :]
print test.shape
spike_source_data = sr.gen_spike_source(test)

(10, 784)


In [58]:
spikes = sr.run_test(w, cell_params_lif, spike_source_data)

In [ ]:
def kb_test(scaled_w, test, SUM_rate=2000.):
    

In [6]:
spike_count = list()
dur_test = 1000
silence = 200
for i in range(w[-1].shape[1]):
    index_i = np.where(spikes[:,0] == i)
    spike_train = spikes[index_i, 1]
    #temp = np.histogram(spike_train, bins=range(0, (dur_test+silence)*num_test+1,dur_test+silence))[0]
    temp = sr.counter(spike_train, range(0, (dur_test+silence)*num_test,dur_test+silence), dur_test)
    spike_count.append(temp)
spike_count = np.array(spike_count)/(dur_test / 1000.)

In [7]:
print spike_count, np.argmax(spike_count, axis=0), dbnet['test_y'][:10]

[[  0.   1.   0.   8.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  88.   0.   0.  74.   0.   0.   0.   0.]
 [  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  24.   0.   9.   2.   0.   2.]
 [  0.   0.   1.   0.   0.   0.   1.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   1.   0.]
 [ 30.   0.   0.   0.   1.   3.   0.   0.   0.   4.]
 [  0.   0.   2.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   6.   0.   0.   1.   0.   4.]] [7 0 1 0 4 1 4 4 5 7] [7 2 1 0 4 1 4 9 5 9]


In [ ]:
mu.plot_digit(test_x[3])

In [54]:
(np.argmax(spike_count, axis=0) == dbnet['test_y'][:10]).astype(int)

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [48]:
result_list[:10, 1]

array([ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.])

In [2]:
rs = np.load('result_list.npy')
limit = 9990.
correct = np.where(rs[:limit,1] == 1)[0].shape[0]
wrong = np.where(rs[:limit,1] == 0)[0].shape[0]
noresp = np.where(rs[:limit,1] == -1)[0].shape[0]
print  correct/limit*100.,  wrong/limit*100., noresp/limit*100.

89.8498498498 8.71871871872 1.43143143143
